In [1]:
import matplotlib.pyplot as fig
import datetime as dt
import pandas as pd
import yfinance as yf
import numpy as np

from contourpy.util.data import simple
from scipy import stats

## importando pelo yahoo finance

# Diretorio dos dados extraidos

D:\ProjetoFII\dadosColetadosFii


In [2]:
jsonFiisImport = pd.read_json(r'C:\Users\Heitor\PycharmProjects\DadosFii\Fii.json')
jsonFiisImport

,title,tipo,descript,date year,date month,date day
0,AAGR11,Fiagro:,ASSET BANK AGRONEGÓCIOS FIAGRO IMOBILIARIO,2024,3,9
1,AAZQ11,Fiagro: Fiagro,None,2024,3,9
2,ABCP11,Tijolo: Shoppings,Grand Plaza Shopping,2024,3,9
3,AFCR11,Outros: Indefinido,None,2024,3,9
4,AFHI11,Papel: Papéis,AF INVEST CRI,2024,3,9
...,...,...,...,...,...,...
582,YEES11,Fundos Imobiliários:,None,2024,3,9
583,YUFI11,Fundo Misto: Fundo de Desenvolvimento,YUCA FDO INV. IMOB.,2024,3,9
584,ZAVC11,Fundos Imobiliários:,None,2024,3,9
585,ZAVI11,Fundo Misto: Indefinido,None,2024,3,9


# Realizando a requisição dos fundos

In [3]:
dataJson = list()

for Fii in jsonFiisImport['title']:
    dataJson.append(yf.Ticker(Fii + '.SA'))
dataJson
    

[yfinance.Ticker object <AAGR11.SA>,
 yfinance.Ticker object <AAZQ11.SA>,
 yfinance.Ticker object <ABCP11.SA>,
 yfinance.Ticker object <AFCR11.SA>,
 yfinance.Ticker object <AFHI11.SA>,
 yfinance.Ticker object <AFOF11.SA>,
 yfinance.Ticker object <AGRX11.SA>,
 yfinance.Ticker object <AIEC11.SA>,
 yfinance.Ticker object <AJFI11.SA>,
 yfinance.Ticker object <ALMI11.SA>,
 yfinance.Ticker object <ALZC11.SA>,
 yfinance.Ticker object <ALZM11.SA>,
 yfinance.Ticker object <ALZR11.SA>,
 yfinance.Ticker object <AMAZ11.SA>,
 yfinance.Ticker object <ANCR11B.SA>,
 yfinance.Ticker object <APTO11.SA>,
 yfinance.Ticker object <APXM11.SA>,
 yfinance.Ticker object <APXR11.SA>,
 yfinance.Ticker object <APXU11.SA>,
 yfinance.Ticker object <AQLL11.SA>,
 yfinance.Ticker object <ARCT11.SA>,
 yfinance.Ticker object <ARFI11B.SA>,
 yfinance.Ticker object <AROA11.SA>,
 yfinance.Ticker object <ARRI11.SA>,
 yfinance.Ticker object <ARXD11.SA>,
 yfinance.Ticker object <ASMT11.SA>,
 yfinance.Ticker object <ASRF11.SA>,

In [4]:
historicInfo = dict()
for funds in dataJson:
    historicInfo.update({funds.info.get('symbol'): funds.history(period="max")})
historicInfo

AGRX11.SA: Period 'max' is invalid, must be one of ['1d', '5d']
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AMAZ11.SA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AMAZ11.SA&crumb=lwJBgOWkYDP
$AMAZ11.SA: possibly delisted; no timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ANCR11B.SA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ANCR11B.SA&crumb=lwJBgOWkYDP
$ANCR11B.SA: possibly delisted; no timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/APXU11.SA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=APXU11.SA&crumb=lwJBgOWkYDP
$APXU11.SA: poss

{'AAGR11.SA':                                 Open       High        Low      Close  Volume  \
 Date                                                                            
 2024-02-19 00:00:00-03:00  87.685684  87.685684  87.685684  87.685684       0   
 2024-02-20 00:00:00-03:00  88.589096  88.589096  88.589096  88.589096       0   
 2024-02-21 00:00:00-03:00  90.341728  90.341728  90.341728  90.341728      87   
 2024-02-22 00:00:00-03:00  90.341728  90.341728  90.341728  90.341728      12   
 2024-02-23 00:00:00-03:00  90.341728  90.341728  90.341728  90.341728      32   
 ...                              ...        ...        ...        ...     ...   
 2024-11-14 00:00:00-03:00  88.000000  94.669998  88.000000  94.660004     283   
 2024-11-18 00:00:00-03:00  94.660004  98.000000  94.660004  97.500000      24   
 2024-11-19 00:00:00-03:00  97.500000  97.500000  94.000000  97.000000      29   
 2024-11-21 00:00:00-03:00  97.500000  97.500000  96.330002  97.500000      29   
 20

# Filtrar todos os fundos que o request na API conseguiu ter acesso aos dados:

### Exemplo de exibição, simpleInfo é uma lista de dataframes

In [5]:
print(len(historicInfo))

495


In [6]:
for stock in historicInfo.values():
    if isinstance(stock.index, pd.core.indexes.datetimes.DatetimeIndex):
        stock.index = stock.index.strftime('%d/%m/%Y') 

In [7]:
historicInfo.values()

dict_values([                 Open       High        Low      Close  Volume  Dividends  \
Date                                                                        
19/02/2024  87.685684  87.685684  87.685684  87.685684       0        0.0   
20/02/2024  88.589096  88.589096  88.589096  88.589096       0        0.0   
21/02/2024  90.341728  90.341728  90.341728  90.341728      87        0.0   
22/02/2024  90.341728  90.341728  90.341728  90.341728      12        0.0   
23/02/2024  90.341728  90.341728  90.341728  90.341728      32        0.0   
...               ...        ...        ...        ...     ...        ...   
14/11/2024  88.000000  94.669998  88.000000  94.660004     283        0.0   
18/11/2024  94.660004  98.000000  94.660004  97.500000      24        0.0   
19/11/2024  97.500000  97.500000  94.000000  97.000000      29        0.0   
21/11/2024  97.500000  97.500000  96.330002  97.500000      29        0.0   
22/11/2024  97.500000  97.500000  97.000000  97.500000      21 

## Filtrar com base na media de volume - Minimo: 1000

In [8]:
stocks_volume_min_filter = [stock for stock in historicInfo.values() if stock['Volume'].mean() >= 1000]
stocks_volume_min_filter

[                Open      High       Low     Close  Volume  Dividends  \
 Date                                                                    
 21/12/2022  7.514310  7.551881  7.514310  7.536852   78854        0.0   
 22/12/2022  7.664595  7.664595  7.138594  7.514309   31942        0.0   
 23/12/2022  7.514309  7.514309  7.364023  7.514309    6389        0.0   
 26/12/2022  7.581938  7.581938  7.401595  7.499280   38590        0.0   
 27/12/2022  7.499279  7.499279  7.063450  7.288879   21828        0.0   
 ...              ...       ...       ...       ...     ...        ...   
 14/11/2024  6.550000  6.640000  6.540000  6.620000  110918        0.0   
 18/11/2024  6.620000  6.660000  6.550000  6.580000  111350        0.0   
 19/11/2024  6.580000  6.720000  6.580000  6.650000  102021        0.0   
 21/11/2024  6.650000  6.730000  6.650000  6.680000   91133        0.0   
 22/11/2024  6.720000  6.740000  6.670000  6.740000   39070        0.0   
 
             Stock Splits  
 Date   

In [9]:
print(len(stocks_volume_min_filter))

278


## Adicionar colunas com a regressão linear, comparando com o valor: b3, IPCA, tesouro direto

In [79]:
ibova = pd.DataFrame(yf.Ticker('BOVA11.SA').history(period="max"))
ibova.index = ibova.index.strftime('%d/%m/%Y')
ibova

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
17/04/2009,45.700001,45.980000,45.599998,45.849998,274700,0.0,0.0
20/04/2009,44.790001,44.900002,44.070000,44.419998,265800,0.0,0.0
22/04/2009,44.610001,45.240002,44.509998,44.990002,234800,0.0,0.0
23/04/2009,45.189999,45.900002,44.860001,45.900002,305900,0.0,0.0
24/04/2009,46.169998,46.849998,46.099998,46.689999,325600,0.0,0.0
...,...,...,...,...,...,...,...
14/11/2024,124.209999,125.050003,124.010002,124.269997,3889073,0.0,0.0
18/11/2024,124.089996,124.900002,123.820000,124.410004,4611947,0.0,0.0
19/11/2024,124.199997,125.199997,123.849998,124.879997,5651357,0.0,0.0


In [95]:
for stock in stocks_volume_min_filter:
    ibova_filtrada = ibova.loc[str(stock.index[0]) : str(stock.index[-1])]
    print(ibova_filtrada.shape, stock.index.shape)
    dados1 = np.array(stock['Close'])
    dados2 = np.array(ibova_filtrada['Close'])
    #beta,beta0,r_value,p_value,std_err = stats.linregress(dados1,dados2)
    #yLin = beta*dados1 + beta0
    #print('%10.6f %10.6f %5.2f %10.6f %10.8f' % (beta, beta0, r_value, p_value, std_err))
    #fig.plot(dados1,yLin, '-k',dados1, dados2, 'ok')
    #fig.xlabel('IBOVESPA')
    #fig.ylabel('Fundo')


(482, 7) (482,)
(3499, 7) (3527,)
(911, 7) (911,)
(676, 7) (676,)
(261, 7) (261,)
(451, 7) (147,)
(919, 7) (899,)
(1694, 7) (1711,)
(676, 7) (676,)
(322, 7) (322,)
(234, 7) (234,)
(1252, 7) (1252,)
(394, 7) (394,)
(1115, 7) (1115,)
(970, 7) (970,)
(704, 7) (704,)
(2538, 7) (2565,)
(2538, 7) (2565,)
(2538, 7) (2565,)
(2994, 7) (2763,)
(1750, 7) (1774,)
(1750, 7) (1774,)
(695, 7) (695,)
(948, 7) (943,)
(1156, 7) (285,)
(64, 7) (64,)
(2538, 7) (2565,)
(1256, 7) (1256,)
(1236, 7) (1236,)
(2538, 7) (2565,)
(787, 7) (229,)
(410, 7) (410,)
(677, 7) (661,)
(950, 7) (950,)
(1453, 7) (1465,)
(1453, 7) (1465,)
(2538, 7) (2565,)
(521, 7) (521,)
(832, 7) (832,)
(974, 7) (974,)
(207, 7) (207,)
(2538, 7) (2565,)
(2537, 7) (2564,)
(459, 7) (459,)
(570, 7) (442,)
(2538, 7) (2565,)
(1008, 7) (848,)
(266, 7) (266,)
(3016, 7) (2971,)
(1220, 7) (1220,)
(347, 7) (347,)
(349, 7) (349,)
(608, 7) (608,)
(2270, 7) (2235,)
(412, 7) (412,)
(1284, 7) (1284,)
(676, 7) (676,)
(655, 7) (655,)
(908, 7) (908,)
(752, 7)

KeyError: '01/03/2018'